# Experiment 01

First Tests from the predictions.

In [5]:
import numpy as np
import pandas as pd
import math

import sys
import Utils 
from Utils import calculateEMA, calculateBB, addPreviousDays, removeColumns, getPeriod, mape, mae, createDate, getDate, printResult

import matplotlib.pyplot as plt
import matplotlib.dates as mpdates
import mplfinance as mpf
from mplfinance.original_flavor import candlestick_ohlc

import warnings
warnings.filterwarnings('ignore')


def getPeriod2(df, begin, end, resetIndex = False):
    """
    Returns the df in the chosen interval
    
    Object begin: Start date forrmated as 'yyyy.mm.dd'.
    Object   end: End date 'yyyy.mm.dd'.

    returns a dataframe with the historic of the selected period
    """
    
    indexBegin = df[df['date']==begin].index[0]
    indexEnd = df[df['date']==end].tail(1).index[0]
    
    if (resetIndex):
        return df[(df.index >= indexBegin) & (df.index <= indexEnd)].reset_index(drop=True)
    else: 
        return df[(df.index >= indexBegin) & (df.index <= indexEnd)]

In [7]:
dfColumns = ['date','time','open', 'high', 'low','close','tickvol','volume','spread']
newColumns = ['open', 'high', 'low', 'close']

TEST_BEGIN = '2021.04.06'
TEST_END = '2021.06.01'

TRAIN_BEGIN = '2020.03.31'
TRAIN_END = '2021.05.31'

TIMES = 10

In [8]:
df = pd.read_csv('../Data/PETR4_M15.csv', sep = "\t")
df.columns = dfColumns
df = removeColumns(df, ['tickvol', 'volume','spread'])

df2 = pd.read_csv('../Data/VALE3_M15.csv', sep = "\t")
df2.columns = dfColumns
df2 = removeColumns(df2, ['tickvol', 'volume','spread'])

initialDf_PETR4 = getPeriod2(df, TEST_BEGIN, TEST_END)
initialDf_VALE3 = getPeriod2(df2, TEST_BEGIN, TEST_END)

In [4]:
initialDf_PETR4

,date,time,open,high,low,close
6197,2021.04.06,10:00:00,23.39,23.47,23.32,23.32
6198,2021.04.06,10:15:00,23.32,23.38,23.30,23.36
6199,2021.04.06,10:30:00,23.36,23.53,23.33,23.52
6200,2021.04.06,10:45:00,23.52,23.58,23.39,23.48
6201,2021.04.06,11:00:00,23.48,23.49,23.36,23.39
...,...,...,...,...,...,...
7311,2021.06.01,15:45:00,27.14,27.14,27.05,27.06
7312,2021.06.01,16:00:00,27.06,27.09,27.01,27.02
7313,2021.06.01,16:15:00,27.01,27.17,27.01,27.14
7314,2021.06.01,16:30:00,27.15,27.31,27.11,27.27


In [9]:
initialDf_VALE3

,date,time,open,high,low,close
6186,2021.04.06,10:00:00,103.80,104.35,103.52,103.77
6187,2021.04.06,10:15:00,103.77,103.80,102.92,103.23
6188,2021.04.06,10:30:00,103.23,104.25,103.00,104.23
6189,2021.04.06,10:45:00,104.23,104.56,103.70,104.03
6190,2021.04.06,11:00:00,104.03,104.12,103.15,103.34
...,...,...,...,...,...,...
7301,2021.06.01,15:45:00,113.84,113.95,113.61,113.72
7302,2021.06.01,16:00:00,113.72,114.00,113.70,113.99
7303,2021.06.01,16:15:00,113.99,114.00,113.90,113.96
7304,2021.06.01,16:30:00,113.97,114.00,113.88,113.88


In [10]:
pred_PETR4 = pd.read_pickle('../Data/PETR4_Pred_with_confidence.pkl')
pred_PETR4.head(5)

,high,low,high_pred,low_pred,uncertainty,date
1299,23.58,23.13,23.624167,23.038600,0.264795,2021.04.06
1300,23.55,23.15,23.563533,23.027933,0.247948,2021.04.07
1301,23.26,22.61,23.551200,23.009967,0.225871,2021.04.08
1302,23.01,22.61,23.154067,22.493967,0.318393,2021.04.09
1303,23.58,23.01,23.675200,22.946867,0.401854,2021.04.12


In [11]:
pred_VALE3 = pd.read_pickle('../Data/VALE3_Pred_with_confidence.pkl')
pred_VALE3.head(15)

,high,low,high_pred,low_pred,uncertainty,date
1299,104.56,102.05,102.036967,98.697267,2.018520,2021.04.06
1300,105.32,102.15,102.133233,98.759433,2.090933,2021.04.07
1301,105.30,104.12,102.025533,98.788033,1.981552,2021.04.08
1302,104.66,102.94,102.055500,98.819167,1.938339,2021.04.09
1303,103.63,102.21,102.029900,98.800400,1.984671,2021.04.12
1304,104.29,102.85,102.029900,98.800400,1.984671,2021.04.13
1305,107.78,103.77,102.029900,98.800400,1.984671,2021.04.14
1306,109.88,107.55,102.029900,98.800400,1.984671,2021.04.15
1307,109.11,107.50,102.029900,98.800400,1.984671,2021.04.16
1308,109.88,107.64,102.029900,98.800400,1.984671,2021.04.19


### Início do sistema

In [12]:
def getDay(df, day):
    return df[df['date']==day]

def buy(value, availableMoney, opr='normal'):
    lotValue = value*100
    quantity = math.floor(availableMoney/lotValue)
    
    amount = quantity*lotValue
    remainingMoney = availableMoney - amount
    
    return remainingMoney, quantity

def sell(value, amount, opr='normal'):
    if(opr == 'normal'):
        return value*amount*100
    else:
        return value*amount*100

In [13]:
def runStrategy(df, dailyDf, Money, stopLoss, std):
    size = len(df.index)

    Money = Money
    stopLoss = stopLoss

    orders = []

    for day in range(0,size):
        currentDay =  df.iloc[day].date

        lowPred = df.iloc[day].low_pred*(1+std)
        highPred = df.iloc[day].high_pred*(1-std)

        dayDf = getPeriod2(dailyDf, currentDay, currentDay)
        daySize = len(dayDf.index)

        op = 'none'
        quantity = 0
        trade = []

        for i in range(0,daySize):
            currentTime = dayDf.iloc[i]

            if((op == 'buy') and (currentTime.close < opPrice*(1-stopLoss) or currentTime.close >= highPred or i == daySize-1)):
                isStop = False
                if(currentTime.close < opPrice*(1-stopLoss)): isStop = True
                aux = sell(currentTime.close, quantity)
                Money = Money + aux 
                trade = trade + [currentTime.time, 'Sell', currentTime.close, currentTime.close-opPrice,(currentTime.close-opPrice)*100*quantity, isStop, Money]
                orders.append(trade)

                quantity = 0
                op = 'none'

            elif((op == 'sell') and (currentTime.close > opPrice*(1+stopLoss) or currentTime.close <= lowPred or i == daySize-1)):
                isStop = False
                if(currentTime.close > opPrice*(1+stopLoss)): isStop = True

                aux = sell(currentTime.close, quantity)
                Money = Money + (opPrice*100*quantity) + ((opPrice*100*quantity) - aux) 
                trade = trade + [currentTime.time, 'Buy', currentTime.close, opPrice-currentTime.close,(opPrice-currentTime.close)*100*quantity, isStop, Money]
                orders.append(trade)

                quantity = 0
                op = 'none'

            elif(op == 'none' and i < daySize-1 ):
                if(currentTime.close <= lowPred):
                    Money, quantity = buy(currentTime.close, Money)
                    opPrice = currentTime.close
                    op = 'buy'

                    trade = [currentTime.date, quantity, currentTime.time, 'Buy', currentTime.close]

                elif(currentTime.close >=  highPred):
                    Money, quantity = buy(currentTime.close, Money)
                    opPrice = currentTime.close
                    op = 'sell'

                    trade = [currentTime.date, quantity, currentTime.time, 'Sell', currentTime.close]
        print('eday '+str(day)+' money: '+str(Money))
    
    ordersDf = pd.DataFrame(orders)
    ordersDf.columns = ['date','batches','time_op1','op1','value_op1','time_op2','op2','value_op2', 'profit', 'totalProfit', 'stop','Amount']
    return ordersDf

In [14]:
testPetr = runStrategy(pred_PETR4, initialDf_PETR4, 500000, 0.02, 0.02)
print('Accuracy: '+str(round(100*len(testPetr[testPetr['profit'] > 0].index)/len(testPetr.index), 2)))

#testVale = runStrategy(pred_VALE3, initialDf_VALE3, 100000, 0.01, 0.02)
#print('Accuracy: '+str(round(100*len(testVale[testVale['profit'] > 0].index)/len(testVale.index), 2)))

eday 0 money: 499776.00000000023
eday 1 money: 504516.0
eday 2 money: 501900.0
eday 3 money: 504958.9999999998
eday 4 money: 507094.9999999999
eday 5 money: 507970.99999999994
eday 6 money: 508616.0
eday 7 money: 496819.0000000002
eday 8 money: 499186.0000000003
eday 9 money: 489940.00000000023
eday 10 money: 482076.0000000003
eday 11 money: 478660.0000000003
eday 12 money: 477257.0000000005
eday 13 money: 477451.00000000047
eday 14 money: 471602.0000000005
eday 15 money: 468406.00000000047
eday 16 money: 463262.0000000005
eday 17 money: 473308.0000000004
eday 18 money: 469113.00000000035
eday 19 money: 458129.00000000035
eday 20 money: 458561.0000000004
eday 21 money: 457001.0000000003
eday 22 money: 449402.0000000003
eday 23 money: 458310.0000000003
eday 24 money: 458833.0000000004
eday 25 money: 469020.00000000047
eday 26 money: 479688.00000000035
eday 27 money: 470000.0000000003
eday 28 money: 470874.0000000002
eday 29 money: 473901.0000000003
eday 30 money: 479195.00000000023
eday

In [20]:
testPetr.tail(10)

,date,batches,time_op1,op1,value_op1,time_op2,op2,value_op2,profit,totalProfit,stop,Amount
81,2021.05.27,198,16:00:00,Buy,25.98,16:15:00,Sell,26.01,0.03,594.0,False,516298.0
82,2021.05.27,198,16:30:00,Buy,25.99,16:45:00,Sell,25.91,-0.08,-1584.0,False,514714.0
83,2021.05.28,194,10:00:00,Sell,26.41,14:30:00,Buy,26.97,-0.56,-10864.0,True,503850.0
84,2021.05.28,186,14:45:00,Sell,26.95,16:45:00,Buy,26.99,-0.04,-744.0,False,503106.0
85,2021.05.31,186,10:15:00,Buy,26.92,10:30:00,Sell,27.00,0.08,1488.0,False,504594.0
86,2021.05.31,187,12:00:00,Buy,26.85,16:45:00,Sell,26.87,0.02,374.0,False,504968.0
87,2021.06.01,184,10:00:00,Buy,27.37,10:15:00,Sell,27.52,0.15,2760.0,False,507728.0
88,2021.06.01,185,10:30:00,Buy,27.36,10:45:00,Sell,27.36,0.00,0.0,False,507728.0
89,2021.06.01,185,11:00:00,Sell,27.42,11:30:00,Buy,27.37,0.05,925.0,False,508653.0
90,2021.06.01,187,11:45:00,Buy,27.18,16:45:00,Sell,27.29,0.11,2057.0,False,510710.0


In [12]:
testVale.tail(10)

,date,batches,time_op1,op1,value_op1,time_op2,op2,value_op2,profit,totalProfit,stop,Amount
49,2021.05.24,9,14:45:00,Sell,109.74,16:45:00,Buy,109.78,-0.04,-36.0,False,104058.0
50,2021.05.25,9,10:00:00,Sell,110.10,16:45:00,Buy,107.05,3.05,2745.0,False,106803.0
51,2021.05.26,10,10:00:00,Sell,106.65,10:30:00,Buy,107.83,-1.18,-1180.0,True,105623.0
52,2021.05.26,9,10:45:00,Sell,108.22,13:45:00,Buy,109.52,-1.30,-1170.0,True,104453.0
53,2021.05.26,9,14:00:00,Sell,109.52,16:45:00,Buy,110.20,-0.68,-612.0,False,103841.0
54,2021.05.27,9,10:00:00,Sell,111.35,16:45:00,Buy,110.99,0.36,324.0,False,104165.0
55,2021.05.28,9,10:00:00,Sell,111.15,14:30:00,Buy,112.70,-1.55,-1395.0,True,102770.0
56,2021.05.28,9,14:45:00,Sell,112.29,16:45:00,Buy,111.59,0.70,630.0,False,103400.0
57,2021.05.31,9,10:00:00,Sell,113.62,16:45:00,Buy,114.78,-1.16,-1044.0,True,102356.0
58,2021.06.01,8,10:00:00,Sell,117.54,16:45:00,Buy,113.20,4.34,3472.0,False,105828.0


In [1]:
df = pred_VALE3
dailyDf = initialDf_VALE3
size = len(df.index)

Money = 100000
stopLoss = 0.01
LastTrade = ['None',10,100]

orders = []

for day in range(0,size):
    currentDay =  df.iloc[day].date

    lowPred = df.iloc[day].low_pred*1.01
    highPred = df.iloc[day].high_pred*0.99
    
    dayDf = getPeriod2(dailyDf, currentDay, currentDay)
    daySize = len(dayDf.index)
    
    op = 'none'
    quantity = 0
    trade = []
    
    print('Begining day '+str(day)+' money: '+str(Money))
    for i in range(0,daySize):
        currentTime = dayDf.iloc[i]
        
        if((op == 'buy') and (currentTime.close < opPrice*(1-stopLoss) or currentTime.close >= highPred or i == daySize-1)):
            isStop = False
            if(currentTime.close < opPrice*(1-stopLoss)): isStop = True
            aux = sell(currentTime.close, quantity)
            Money = Money + aux 
            trade = trade + [currentTime.time, 'Sell', currentTime.close, currentTime.close-opPrice,(currentTime.close-opPrice)*100*quantity, isStop, Money]
            orders.append(trade)
  
            quantity = 0
            op = 'none'
        
        elif((op == 'sell') and (currentTime.close > opPrice*(1+stopLoss) or currentTime.close <= lowPred or i == daySize-1)):
            isStop = False
            if(currentTime.close > opPrice*(1+stopLoss)): isStop = True

            aux = sell(currentTime.close, quantity)
            Money = Money + (opPrice*100*quantity) + ((opPrice*100*quantity) - aux) 
            trade = trade + [currentTime.time, 'Buy', currentTime.close, opPrice-currentTime.close,(opPrice-currentTime.close)*100*quantity, isStop, Money]
            orders.append(trade)
            
            quantity = 0
            op = 'none'
        
        elif(op == 'none' and i < daySize-1 ):
            if(currentTime.close <= lowPred):
                Money, quantity = buy(currentTime.close, Money)
                opPrice = currentTime.close
                op = 'buy'
                
                trade = [currentTime.date, quantity, currentTime.time, 'Buy', currentTime.close]
                
            elif(currentTime.close >=  highPred):
                Money, quantity = buy(currentTime.close, Money)
                opPrice = currentTime.close
                op = 'sell'
                
                trade = [currentTime.date, quantity, currentTime.time, 'Sell', currentTime.close]

test2 = pd.DataFrame(orders)
test2.columns = ['date','batches','time_op1','op1','value_op1','time_op2','op2','value_op2', 'profit', 'totalProfit', 'stop','Amount']
test2.tail(60)

test2 = pd.DataFrame(orders)
test2.columns = ['date','batches','time_op1','op1','value_op1','time_op2','op2','value_op2', 'profit', 'totalProfit', 'stop','Amount']

print('Accuracy: '+str(round(100*len(test2[test2['profit'] > 0].index)/len(test2.index), 2)))
test2.head(10)

NameError: name 'pred_VALE3' is not defined